# Вариант № 4
## Задача 2
## А) Подобрать такую маршрутную матрицу, чтобы пропускная способность сети была максимальной.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math as m

In [2]:
def get_omega(L, theta):
    omega = np.array([1/(L+1) for _ in range(L+1)]) 
    for _ in range(1000):
        n_omega = omega @ theta
        omega = n_omega
    return omega

In [3]:
def get_lambda(L, l0, omega):
    lambdas = np.zeros(L) # интенсивности приходов в системы
    for i in range(1, len(omega)):
        lambdas[i-1] = l0*(omega[i]/omega[0])
    return lambdas

In [4]:
def get_psi(lambdas, k, mu):
    psi = np.zeros(L) # коэффициенты использования систем
    for i in range(L):
        psi[i] = lambdas[i]/(k[i]*mu[i])
    return psi

In [5]:
def get_tau(L, l0, k, psy, lambdas):
    Pi0 = np.zeros(L)
    bi = np.zeros(L)
    hi = np.zeros(L)
    ni = np.zeros(L)
    ui = np.zeros(L)
    tau = 0
    for i in range(L):
        _sum = 0
        for j in range(int(k[i])):
            _sum += (k[i]*psy[i])**j / m.factorial(j)
        Pi0[i] = ((((k[i]*psy[i])**k[i])/
                       (m.factorial(k[i])*(1-psy[i])))+_sum)**(-1)
        bi[i] = Pi0[i]*(((k[i]**k[i])*(psy[i]**(k[i]+1)))/
                            (m.factorial(k[i])*((1-psy[i])**2)))
        hi[i] = psy[i]*k[i]
        ni[i] = bi[i]+hi[i]
        ui[i] = ni[i]/lambdas[i]
        tau += lambdas[i]*ui[i]
    tau = (1/l0)*tau
    
    return tau

In [6]:
L = 5 # число приборов
l0 = 0.5 # интенсивность поступления
k = np.ones(L) # количество приборов в i-ой системе
mu= np.ones(L) # интенсивности обслуживания в i-ой системе

theta = np.array([
    [0,       0.7,    0,      0.3,    0,      0],
    [0,       0,      0.6,    0,      0.4,    0],
    [0,       0,      0.5,    0,      0,      0.5],
    [0,       0,      0,      0,      1,      0],
    [0.1,     0,      0,      0,      0,      0.9],
    [1,       0,      0,      0,      0,      0]
]) #начальная маршрутная матрица

d = 0.1
leng_arr = np.arange(0.1,1,d)
mmax_start = 0
mmax_end = 0

for i0 in leng_arr:
    for i1 in leng_arr:
        for i2 in leng_arr:
            for i4 in leng_arr:
                                
                theta[0,1] = i0
                theta[0,3] = 1-i0
                                
                theta[1,2] = i1
                theta[1,4] = 1-i1
                                
                theta[2,2] = i2
                theta[2,5] = 1-i2

                theta[4,0] = i4
                theta[4,5] = 1-i4
                                
                omega = get_omega(L, theta)
                templ = get_lambda(L, l0, omega)
                tempp = get_psi(templ, k, mu)
                if (all(tempp < 1)) and (sum(templ) > mmax_start):
                    mmax_start = sum(templ)
                    m_lambdas = np.copy(templ)
                    th_start = np.copy(theta)
                elif (any(tempp > 1)) and (sum(templ) > mmax_end):
                    mmax_end = sum(templ)
                    m_lambdas_start = np.copy(templ)
                    th_end = np.copy(theta)

print('Изначальная пропускная способность сети:', mmax_start)
print('Маршрутная матрица:\n', th_start)
print('\n\n')
print('Максимальная пропускная способность:', mmax_end)
print('Маршрутная матрица:\n', th_end)

Изначальная пропускная способность сети: 2.32
Маршрутная матрица:
 [[0.  0.4 0.  0.6 0.  0. ]
 [0.  0.  0.5 0.  0.5 0. ]
 [0.  0.  0.9 0.  0.  0.1]
 [0.  0.  0.  0.  1.  0. ]
 [0.2 0.  0.  0.  0.  0.8]
 [1.  0.  0.  0.  0.  0. ]]



Максимальная пропускная способность: 5.1355
Маршрутная матрица:
 [[0.  0.9 0.  0.1 0.  0. ]
 [0.  0.  0.9 0.  0.1 0. ]
 [0.  0.  0.9 0.  0.  0.1]
 [0.  0.  0.  0.  1.  0. ]
 [0.1 0.  0.  0.  0.  0.9]
 [1.  0.  0.  0.  0.  0. ]]


## Б) Необходимо подобрать такой вектор интенсивностей обслуживания, чтобы сумма всех его компонент была минимальной, а время реакции было равно Q.

In [7]:
L = 5 # число приборов
l0 = 0.5 # интенсивность поступления
k = np.ones(L) # количество приборов в i-ой системе
mu= np.ones(L) # интенсивности обслуживания в i-ой системе
mu = mu/10
Q = 4

theta = np.array([
    [0,       0.7,    0,      0.3,    0,      0],
    [0,       0,      0.6,    0,      0.4,    0],
    [0,       0,      0.5,    0,      0,      0.5],
    [0,       0,      0,      0,      1,      0],
    [0.1,     0,      0,      0,      0,      0.9],
    [1,       0,      0,      0,      0,      0]
]) #начальная маршрутная матрица

omega = get_omega(L, theta)
lambdas = get_lambda(L, l0, omega)
psi = get_psi(lambdas, k, mu)
tau = get_tau(L, l0, k, psi, lambdas)
index = 0

# приближаемся к значению tau = Q
while m.fabs(tau - Q) > 0.01:
    # меняем элементы вектора по очереди на 0.01
    mu[index % 5] += 0.01
    index += 1
    psi = get_psi(lambdas, k, mu)
    tau = get_tau(L, l0, k, psi, lambdas)

print("Вектор mu: ", mu)
print(f"Сумма вектора mu = {sum(mu)}")
print('Q = ', Q)
print("Время реакции: ", tau)
print("Маршрутная матрица:\n", theta)

Вектор mu:  [1.23 1.22 1.22 1.22 1.22]
Сумма вектора mu = 6.110000000000004
Q =  4
Время реакции:  4.0071611937454525
Маршрутная матрица:
 [[0.  0.7 0.  0.3 0.  0. ]
 [0.  0.  0.6 0.  0.4 0. ]
 [0.  0.  0.5 0.  0.  0.5]
 [0.  0.  0.  0.  1.  0. ]
 [0.1 0.  0.  0.  0.  0.9]
 [1.  0.  0.  0.  0.  0. ]]
